<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/%EC%9C%A0%EC%A7%80%EC%9B%90_%EB%B2%84%EC%8A%A4_%EC%9A%B4%ED%96%89%EC%8B%9C%EA%B0%84_%EC%98%88%EC%B8%A1_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. import libraries & data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [5]:
import pandas as pd
import lightgbm as lgb
import folium

In [7]:
train = pd.read_csv('/content/drive/MyDrive/KaggleStudy/DACON 버스 운행시간 예측 경진대회/train.csv')
test = pd.read_csv('/content/drive/MyDrive/KaggleStudy/DACON 버스 운행시간 예측 경진대회/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/KaggleStudy/DACON 버스 운행시간 예측 경진대회/submission_제출양식.csv')

In [ ]:
print("train: ", train.shape)
train.head()

In [ ]:
print("test: ", test.shape)
test.head()

In [ ]:
submission.head()

# 2. EDA

In [9]:
train.isna().sum()

id                  0
date                0
route_id            0
vh_id               0
route_nm            0
now_latitude        0
now_longitude       0
now_station         0
now_arrive_time     0
distance            0
next_station        0
next_latitude       0
next_longitude      0
next_arrive_time    0
dtype: int64

In [10]:
train[['route_id','next_arrive_time']].groupby('route_id').mean()
# 해당 노선의 버스가 다음 정거장에 도착하는 시간

,next_arrive_time
route_id,
405136001,92.063688
405136002,108.270852
405136007,94.172947
405136012,111.322633
405136521,114.673939
405136522,101.339188
405320111,68.901710
405320112,67.602392
405320113,68.403649


- max: 405136521 (114.673939)     
- min: 405320122 (63.955199)

## folium.Map()
- location 옵션에 있는 좌표를 중심으로 지도 시각화

In [11]:
coord = [35.1559361, 129.1384361]
map_osm = folium.Map(location=coord) # 중심부

map_osm

In [12]:
map_osm = folium.Map(location=coord,zoom_start=11)
map_osm

In [13]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain')
map_osm

In [14]:
map_osm = folium.Map(location=coord, tiles = 'Stamen Toner')
map_osm

In [16]:
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location=coord, zoom_start=12)

folium.Marker(coord, popup='부산대학교', tooltip='부산대학교 tooltip').add_to(map_osm)

map_osm.save('index.html')

map_osm

가장 arrive time이 긴 노선은 제주 시내를 다님 -> 정체로 인함

In [20]:
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route = map_data[map_data['route_id'] == 405136521].drop_duplicates("now_station")

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)

for item in map_bus_route.index:
  lat = map_bus_route.loc[item,'now_latitude']
  long = map_bus_route.loc[item,'now_longitude']

  folium.Marker([lat,long], popup=map_bus_route.loc[item,'now_station'],
                icon = folium.Icon(color='red', icon='info-sign')).add_to(map_osm)

map_osm

In [23]:
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route = map_data[map_data['route_id'] == 405320122].drop_duplicates("now_station")

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)

for item in map_bus_route.index:
  lat = map_bus_route.loc[item,'now_latitude']
  long = map_bus_route.loc[item,'now_longitude']

  folium.Marker([lat,long], popup=map_bus_route.loc[item,'now_station'],
                icon = folium.Icon(color='red', icon='info-sign')).add_to(map_osm)

map_osm

# 3. 데이터 전처리

### 'now_arrive_time' 실수형으로

In [24]:
train['now_arrive_time'] # object -> 문자열

0         06시
1         06시
2         06시
3         06시
4         07시
         ... 
210452    21시
210453    21시
210454    21시
210455    21시
210456    21시
Name: now_arrive_time, Length: 210457, dtype: object

In [27]:
train['now_arrive_time'] = train['now_arrive_time'].str.slice(stop=-1).astype('int')

In [38]:
test['now_arrive_time'] = test['now_arrive_time'].str.slice(stop=-1).astype('int')

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 22.5+ MB


In [39]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  int64  
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(4), object(4)
memory usage: 9.1+ MB


# 4. 변수 선택 및 모델 구축

In [41]:
features = ['now_latitude','now_longitude','now_arrive_time','distance']
target = ['next_arrive_time']

In [42]:
train_X, test_X, train_Y = train[features], test[features], train[target]

# 5. 모델 학습 및 검증

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [33]:
model_dict = {
    'linear': LinearRegression(),
    'rf': RandomForestRegressor(random_state=0,n_jobs=-1),
    'lgbm': lgb.LGBMRegressor(random_state=0)    
}

In [34]:
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [35]:
model_dict['linear']

LinearRegression()

In [36]:
for key in model_dict.keys():
  print(key)

linear
rf
lgbm


In [43]:
model_result = {}

for key in model_dict.keys():
  
  print('###### 훈련 중 #####')
  model_dict[key].fit(train_X,train_Y)

  print('###### 예측 중 #####')
  model_result[key] = model_dict[key].predict(test_X)

###### 훈련 중 #####
###### 예측 중 #####
###### 훈련 중 #####


<ipython-input-43-78d9d411414e>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(train_X,train_Y)


###### 예측 중 #####
###### 훈련 중 #####
###### 예측 중 #####


In [44]:
model_result

{'linear': array([[ 80.75162733],
        [ 86.59637449],
        [ 96.03730695],
        ...,
        [ 93.13399549],
        [ 72.93764136],
        [103.28646517]]),
 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
         31.84467172,  88.46184286]),
 'lgbm': array([ 42.12400643,  60.24878305, 120.81241189, ..., 105.81387439,
         30.82952485, 111.66787953])}

In [46]:
model_result['linear']

array([[ 80.75162733],
       [ 86.59637449],
       [ 96.03730695],
       ...,
       [ 93.13399549],
       [ 72.93764136],
       [103.28646517]])

In [45]:
lr_submit = submission.copy()
rf_submit = submission.copy()
lgbm_submit = submission.copy()

In [48]:
lr_submit['next_arrive_time'] = model_result['linear']
rf_submit['next_arrive_time'] = model_result['rf']
lgbm_submit['next_arrive_time'] = model_result['lgbm']

In [50]:
lr_submit.to_csv('lr_submit.csv', index = False)
rf_submit.to_csv('rf_submit.csv', index = False)
lgbm_submit.to_csv('lgbm_submit.csv', index = False)